In [1]:
import duckdb
from pathlib import Path
from settings import settings
from alpha_vantage_schema import TABLE_SCHEMAS

In [2]:
conn = duckdb.connect(Path(settings.get("db_path", "data/alpha_vantage.db")))

In [3]:
result = conn.execute("SELECT * FROM OVERVIEW").df()
print(result)

    Symbol     AssetType                              Name  \
0      MMM  Common Stock                        3M Company   
1      AOS  Common Stock              Smith AO Corporation   
2      ABT  Common Stock               Abbott Laboratories   
3     ABBV  Common Stock                        AbbVie Inc   
4      ACN  Common Stock                     Accenture plc   
..     ...           ...                               ...   
452    WST  Common Stock  West Pharmaceutical Services Inc   
453     WU  Common Stock                  Western Union Co   
454     WY  Common Stock              Weyerhaeuser Company   
455    WMB  Common Stock            Williams Companies Inc   
456   WYNN  Common Stock              Wynn Resorts Limited   

                                           Description      CIK Exchange  \
0    3M Company is a diversified American multinati...    66740     NYSE   
1    A. O. Smith Corporation stands as a leading Am...    91142     NYSE   
2    Abbott Laboratories (A

In [4]:
conn.close()